In [1]:
from gensim.models import Word2Vec
import sys
from sklearn.linear_model import SGDClassifier
from sklearn.dummy import DummyClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import KFold, cross_val_score
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn import tree
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

Using TensorFlow backend.


In [9]:
# load the model containing features
node2vecmodel = Word2Vec.load_word2vec_format("modelUndirectedClickRate.bin" , binary=False)

In [31]:
X = []
Y = []

def handle_features(source, target):
    features_source = node2vecmodel[source]
    features_target = node2vecmodel[target]
    return features_source * features_target

# load all training pairs
i = 0
with open("/Users/simon/Documents/P7/scripts/trainingPairs.txt") as f:
    for line in f:
        source, target, label = tuple(line.split("\t"))

        if source in node2vecmodel and target in node2vecmodel:
            features = handle_features(source, target)
            label = -1 if int(label) == 0 else 1
            Y.append(label)
            X.append(features)
        else:
            i += 1


print(len([x for x in Y if x == -1]))
print(len([x for x in Y if x == 1]))
print(i)
X = np.array(X)
Y = np.array(Y)

60602
75516
27538


In [32]:
model = SGDClassifier(loss="hinge", penalty="l2")
model.fit(X,Y)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [33]:
model.predict(handle_features("Hull_House", "Dorset"))

/usr/local/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([1])

In [35]:
X[0]

array([  5.01808063e-05,   5.58966394e-06,  -1.74713841e-05,
         1.42608958e-04,  -1.85299668e-05,  -6.68976718e-05,
         1.49306402e-06,  -2.24485571e-04,   7.31448454e-05,
         4.30349392e-05,  -1.58169587e-05,  -2.95010705e-05,
        -1.60777618e-05,   3.43087049e-05,  -2.83456011e-05,
         2.17078014e-05,   8.29275305e-05,  -2.52420985e-04,
         1.88621998e-05,   1.21838912e-05,  -5.20617141e-05,
        -4.38245988e-05,   2.28046410e-05,   1.72204076e-04,
         9.43936029e-05,  -5.68757278e-05,  -1.15217517e-05,
         1.15350376e-04,  -2.58629984e-06,  -1.08805179e-04,
        -1.77248876e-05,  -5.45173425e-05,  -2.52932805e-05,
        -2.26638091e-04,  -1.14771728e-05,  -8.54372047e-05,
         6.97579162e-06,   5.81959430e-05,   3.97224721e-05,
        -3.22274573e-05,   2.00094510e-05,  -5.62878158e-05,
         1.07850537e-05,   9.35170174e-05,  -5.17439958e-07,
         2.09943086e-04,  -2.37368313e-05,  -2.32820275e-05,
        -1.61187327e-05,

In [34]:
num_folds = 10
seed = 7
# prepare models
models = []
models.append(('SGD', SGDClassifier(loss="hinge", penalty="l2")))

# evaluate each model in turn
results = []
names = []
scoring = 'f1'
for name, model in models:
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state=seed)
    cv_results = cross_val_score(model, X, y=Y, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

SGD: 0.713646 (0.001293)
